In [26]:
import tensorflow as tf
from keras.applications.vgg19 import VGG19, preprocess_input
from keras.preprocessing.image import ImageDataGenerator
from keras.preprocessing.image import load_img
from keras.preprocessing.image import img_to_array
from keras.layers import GlobalAveragePooling2D, Dense, Dropout, Flatten
from keras.layers.convolutional import Conv2D
from keras.layers.pooling import MaxPooling2D
from keras.models import Model
from keras import optimizers 
from keras import regularizers
from keras.callbacks import ModelCheckpoint,EarlyStopping,ReduceLROnPlateau
from time import time

from numpy.random import seed
seed(12)
from tensorflow import set_random_seed
set_random_seed(12)

import matplotlib.pyplot as plt
import numpy as np
np.random.seed(12)
from scipy import stats
from sklearn import metrics
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt


from sklearn.metrics import classification_report, f1_score, accuracy_score
import glob
import warnings
import pandas as pd
import numpy as np

import os

## Load dataset

In [27]:
path_dataset = "Dataset/WISDM/WISDM_ar_v1.1/a.txt"

In [28]:
columns = ['user','activity','timestamp', 'x-axis', 'y-axis', 'z-axis']
df = pd.read_csv(path_dataset, header = None, names = columns )
df

,user,activity,timestamp,x-axis,y-axis,z-axis
0,33,Jogging,49105962326000,-0.694638,12.680544,0.503953
1,33,Jogging,49106062271000,5.012288,11.264028,0.953424
2,33,Jogging,49106112167000,4.903325,10.882658,-0.081722
3,33,Jogging,49106222305000,-0.612916,18.496431,3.023717
4,33,Jogging,49106332290000,-1.184970,12.108489,7.205164
5,33,Jogging,49106442306000,1.375655,-2.492524,-6.510526
6,33,Jogging,49106542312000,-0.612916,10.569390,5.706926
7,33,Jogging,49106652389000,-0.503953,13.947236,7.055340
8,33,Jogging,49106762313000,-8.430995,11.413852,5.134871
9,33,Jogging,49106872299000,0.953424,1.375655,1.648062


In [29]:
null_counts = df.isnull().sum()
print("Number of null values in each column:\n{}".format(null_counts))

Number of null values in each column:
user         0
activity     0
timestamp    0
x-axis       0
y-axis       0
z-axis       1
dtype: int64


In [30]:
df = df.dropna()

In [31]:
N_TIME_STEPS = 200
N_FEATURES = 3
step = 20
segments = []
labels = []
for i in range(0, len(df) - N_TIME_STEPS, step):
    xs = df['x-axis'].values[i: i + N_TIME_STEPS]
    ys = df['y-axis'].values[i: i + N_TIME_STEPS]
    zs = df['z-axis'].values[i: i + N_TIME_STEPS]
    label = stats.mode(df['activity'][i: i + N_TIME_STEPS])[0][0]
    segments.append([xs, ys, zs])
    labels.append(label)

C:\Users\fifamd\Anaconda3\envs\tf\lib\site-packages\scipy\stats\stats.py:253: RuntimeWarning: The input array could not be properly checked for nan values. nan values will be ignored.
  "values. nan values will be ignored.", RuntimeWarning)


In [32]:
reshaped_segments = np.asarray(segments, dtype= np.float32).reshape(-1, N_TIME_STEPS, N_FEATURES)
labels = np.asarray(pd.get_dummies(labels), dtype = np.float32)

In [33]:
RANDOM_SEED = 42

In [34]:
X_train, X_test, y_train, y_test = train_test_split(reshaped_segments, labels, test_size=0.2, random_state=RANDOM_SEED)

## Transfer

In [35]:
from keras.models import Model
from keras.layers import Input, Dense,LSTM
from keras.layers.convolutional import Conv1D
from keras.layers.pooling import MaxPooling1D
from keras import optimizers 
from keras import regularizers
from keras.callbacks import ModelCheckpoint,EarlyStopping,ReduceLROnPlateau
from numpy.random import seed
seed(12)
from tensorflow import set_random_seed
set_random_seed(12)

import matplotlib.pyplot as plt
import numpy as np
np.random.seed(12)

from sklearn.metrics import classification_report, f1_score, accuracy_score

In [36]:
def f1_score_metric(y_true, y_pred):
    y_true = tf.cast(y_true, "int32")
    y_pred = tf.cast(tf.round(y_pred), "int32") # implicit 0.5 threshold via tf.round
    y_correct = y_true * y_pred
    sum_true = tf.reduce_sum(y_true, axis=1)
    sum_pred = tf.reduce_sum(y_pred, axis=1)
    sum_correct = tf.reduce_sum(y_correct, axis=1)
    precision = sum_correct / sum_pred
    recall = sum_correct / sum_true
    f_score = 5 * precision * recall / (4 * precision + recall)
    f_score = tf.where(tf.is_nan(f_score), tf.zeros_like(f_score), f_score)
    return tf.reduce_mean(f_score)

In [37]:
print(X_train.shape)
print(y_train.shape)
print(reshaped_segments.shape)
numOfRows = reshaped_segments.shape[1]
numOfColumns = reshaped_segments.shape[2]
filters = 128
Epochs = 10
batchSize = 10
num_class = 6
print(numOfRows , numOfColumns)
print(y_train)

(43920, 200, 3)
(43920, 6)
(54901, 200, 3)
200 3
[[ 0.  0.  0.  1.  0.  0.]
 [ 0.  1.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  1.  0.]
 ..., 
 [ 0.  1.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  1.]
 [ 1.  0.  0.  0.  0.  0.]]


In [17]:
y_train = np.reshape(y_train,(43920,6))

NameError: name 'y' is not defined

In [38]:
from keras.models import Sequential
from keras.layers import Activation, Dense

In [39]:
def TCNN():
    base_model1 = Input(shape=(numOfRows,numOfColumns))
    base_model = Conv1D(filters,(15),activation = 'relu')(base_model1)
    base_model = MaxPooling1D(pool_size=4)(base_model)
    base_model = Conv1D(filters,(15), activation = 'relu')(base_model)
    base_model = MaxPooling1D(pool_size=5)(base_model)

    base_model = LSTM(128)(base_model)

    base_model = Dense(256)(base_model)
    base_model = Dropout(0.2)(base_model)
    base_model = Dense(6 ,activation="softmax")(base_model)
    model = Model(inputs=[base_model1], outputs=base_model)
    adam = optimizers.Adam(lr = 0.001, decay=1e-6)
    model.compile(loss='categorical_crossentropy', optimizer=adam, metrics=['accuracy'])
    return model

In [40]:
m = TCNN()
m.summary()

# Callbacks
reduce_lr = ReduceLROnPlateau(monitor='val_acc', mode='max', factor=0.1, patience=10,
                                verbose=1, cooldown=5, min_lr=0)
early_stopper = EarlyStopping(monitor='val_acc', min_delta=0, 
                           patience=20, verbose=0, mode='max')

model_1_path = "./model/model_1/{}.h5".format("model_1")
checkpoint = ModelCheckpoint(model_1_path, monitor='val_acc', verbose=1,save_best_only=True,save_weights_only=False, mode='max',period=1)
callbacks_list = [checkpoint,early_stopper,reduce_lr]

history = m.fit(X_train,y_train,epochs = Epochs, callbacks=callbacks_list, verbose =1)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 200, 3)            0         
_________________________________________________________________
conv1d_1 (Conv1D)            (None, 186, 128)          5888      
_________________________________________________________________
max_pooling1d_1 (MaxPooling1 (None, 46, 128)           0         
_________________________________________________________________
conv1d_2 (Conv1D)            (None, 32, 128)           245888    
_________________________________________________________________
max_pooling1d_2 (MaxPooling1 (None, 6, 128)            0         
_________________________________________________________________
lstm_1 (LSTM)                (None, 128)               131584    
_________________________________________________________________
dense_3 (Dense)              (None, 256)               33024     
__________

C:\Users\fifamd\Anaconda3\envs\tf\lib\site-packages\keras\callbacks.py:406: RuntimeWarning: Can save best model only with val_acc available, skipping.
  'skipping.' % (self.monitor), RuntimeWarning)
C:\Users\fifamd\Anaconda3\envs\tf\lib\site-packages\keras\callbacks.py:497: RuntimeWarning: Early stopping conditioned on metric `val_acc` which is not available. Available metrics are: loss,acc
  (self.monitor, ','.join(list(logs.keys()))), RuntimeWarning
C:\Users\fifamd\Anaconda3\envs\tf\lib\site-packages\keras\callbacks.py:898: RuntimeWarning: Reduce LR on plateau conditioned on metric `val_acc` which is not available. Available metrics are: loss,acc,lr
  (self.monitor, ','.join(list(logs.keys()))), RuntimeWarning


43920/43920 [==============================] - 78s 2ms/step - loss: 0.1553 - acc: 0.9467
Epoch 3/10
43920/43920 [==============================] - 76s 2ms/step - loss: 0.1157 - acc: 0.9612
Epoch 4/10
43920/43920 [==============================] - 77s 2ms/step - loss: 0.1015 - acc: 0.9654
Epoch 5/10
43920/43920 [==============================] - 78s 2ms/step - loss: 0.0816 - acc: 0.9717
Epoch 6/10
43920/43920 [==============================] - 77s 2ms/step - loss: 0.0730 - acc: 0.9753
Epoch 7/10
43920/43920 [==============================] - 78s 2ms/step - loss: 0.0680 - acc: 0.9774
Epoch 8/10
43920/43920 [==============================] - 77s 2ms/step - loss: 0.0653 - acc: 0.9778
Epoch 9/10
43920/43920 [==============================] - 76s 2ms/step - loss: 0.0629 - acc: 0.9785
Epoch 10/10
43920/43920 [==============================] - 80s 2ms/step - loss: 0.0596 - acc: 0.9806


In [41]:
m.save_weights("./model/model_1.2.h5")

In [90]:
print(m.get_weights())

[array([[[-0.08251053,  0.00512579, -0.04972309, ..., -0.11930908,
          0.10253114, -0.11863396],
        [-0.12347891,  0.03392795,  0.01903475, ..., -0.02300153,
         -0.16131341, -0.17577523],
        [ 0.02494622, -0.00483196, -0.04243604, ...,  0.04696309,
         -0.10071559, -0.11098851]],

       [[ 0.11216159, -0.02476394, -0.05713003, ...,  0.03858734,
          0.00717816,  0.06111021],
        [-0.195917  , -0.03960482, -0.03077825, ..., -0.06645834,
          0.07262053, -0.00341955],
        [-0.0092715 , -0.12777041, -0.05769717, ..., -0.17838943,
         -0.22541054, -0.10149846]],

       [[-0.08551205, -0.15963791, -0.05136813, ..., -0.10585863,
          0.06197259,  0.07183219],
        [ 0.01129385, -0.1321162 , -0.07725766, ...,  0.09558994,
          0.02372855,  0.04819582],
        [ 0.04580235,  0.04125176, -0.11423775, ...,  0.15815488,
          0.21372306,  0.03404731]],

       ..., 
       [[-0.08390899, -0.007429  ,  0.02613281, ...,  0.006734

In [42]:
%run load_dataset.ipynb

Loading 1001's data
Loading 1002's data
Loading 1003's data
Loading 1004's data
Loading 1005's data
Loading 1006's data
Loading 1007's data
Loading 1008's data
Loading 1009's data
Loading 1010's data
Loading 1011's data
Loading 1012's data
Loading 2001's data
Loading 2002's data
Finished loading


In [43]:
%run preprocess_for_SVM.ipynb

In [44]:
subj_train = ['1001','1002','1003','2001']
subj_val = ['1004','1005','1006','2002']
subj_test = ['1007','1008','1009']

In [45]:
def get_X_y(subj_ids, X, y, s):
    X_get = []
    y_get = []
    s_get = []
    for i in range(len(s)):
        for j in range(len(subj_ids)):
            if(s[i]==subj_ids[j]):
                X_get.append(X[i])
                y_get.append(y[i])
                s_get.append(s[i])
                
    return np.array(X_get), np.array(y_get), np.array(s_get)

In [46]:
X_train, y_train, s_train = get_X_y(subj_train, X_all, y_all, subj_all)
X_val, y_val, s_val = get_X_y(subj_val, X_all, y_all, subj_all)
X_test, y_test, s_test = get_X_y(subj_test, X_all, y_all, subj_all)

In [47]:
def prepare_pure(X, y, subj_all, new_label_list):
    X_label, y_label = label_grouping(X, y, subj_all, new_label_list)
    
    X_concat = []
    y_concat = []
    for i in range(len(X_label)):
        for j in range(len(X_label[i])):
            X_ol, y_ol = make_overlapping(np.array(X_label[i][j]), y_label[i][j])
#             print(i, j, X_ol.shape)
            
            if(len(X_concat)==0):
                X_concat = X_ol
            else:
                X_concat = np.vstack((X_concat, X_ol))

            if(len(y_concat)==0):
                y_concat = y_ol
            else:
                y_concat = np.hstack((y_concat, y_ol))
                
#     X_concat_xyz = concat_xyz(X_concat)
    
    return X_concat, y_concat

In [48]:
X_label, y_label = label_grouping(X_train, y_train, s_train, label_list)

In [49]:
label_list = [0,1,3,2,4]
label_dict = {
    0: 'sit',
    1: 'sleep',
    3: 'stand',
    2: 'stairs',
    4: 'walk'
}

all_subjects = subj_train
X_train_pure, y_train_pure = prepare_pure(X_train, y_train, s_train, label_list)

all_subjects = subj_val
X_val_pure, y_val_pure = prepare_pure(X_val, y_val, s_val, label_list)

all_subjects = subj_test
X_test_pure, y_test_pure = prepare_pure(X_test, y_test, s_test, label_list)

In [50]:
VAL_LEN = X_val_pure.shape[0]
X_train_pure = X_train_pure[:VAL_LEN]
y_train_pure = y_train_pure[:VAL_LEN]

print(VAL_LEN)

6534


In [51]:
LABEL_COUNT = 5

In [52]:
def reshape_y(y, window_length=60):
    
    y_rs_ = label_binarize(y, classes=[0,1,2,3,4])
    
    y_rs = []
    for i in range(len(y_rs_)):
        y_temp = []
        for j in range(window_length):
            y_temp.append(y_rs_[i])
            
        y_rs.append(np.array(y_temp))
        
    y_rs = np.array(y_rs)
    y_rs = y_rs.reshape((y_rs.shape[0],y_rs.shape[1],LABEL_COUNT))
        
    return np.array(y_rs)

In [53]:
def reshape_X(X):
    X_rs = X.reshape((X.shape[0],1,X.shape[1]))
    return X_rs

In [54]:
# Y_train = np_utils.to_categorical(y_train, nb_classes)

In [55]:
y_train_rs = reshape_y(y_train_pure)
y_val_rs = reshape_y(y_val_pure)
y_test_rs = reshape_y(y_test_pure)

In [56]:
print(X_train_pure.shape)
print(X_train_pure.shape[1])
print(X_train_pure.shape[2])

(6534, 60, 3)
60
3


In [83]:
%run RNN_model.ipynb

In [70]:
rnn_model = create_rnn_model(X_train_pure)
rnn_model.summary()

ValueError: Input 0 is incompatible with layer gru_8: expected ndim=3, found ndim=2

In [84]:
lstm = lstm_model(X_train_pure)
lstm.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_15 (LSTM)               (None, 60, 50)            10800     
_________________________________________________________________
dropout_12 (Dropout)         (None, 60, 50)            0         
_________________________________________________________________
lstm_16 (LSTM)               (None, 100)               60400     
_________________________________________________________________
dropout_13 (Dropout)         (None, 100)               0         
_________________________________________________________________
dense_14 (Dense)             (None, 256)               25856     
_________________________________________________________________
dropout_14 (Dropout)         (None, 256)               0         
_________________________________________________________________
dense_15 (Dense)             (None, 128)               32896     
__________

In [92]:
lstm.load_weights("./model/model_1.2.h5")

ValueError: Layer #0 (named "lstm_15" in the current model) was found to correspond to layer conv1d_1 in the save file. However the new layer lstm_15 expects 3 weights, but the saved weights have 2 elements.

In [ ]:
compile_model(lstm_model)
train_model(lstm_model, X_train_pure, y_train_rs, 
                        X_val_pure, y_val_rs)

In [ ]:
y_pred = get_y_predict(rnn_model, X_test_pure)

In [ ]:
def convert_y(y):
    y_conv = [j for i in range(len(y)) for j in range(len(y[i][0])) if y[i][0][j]==np.amax(y[i][0])]
    return np.array(y_conv)

In [ ]:
y_t = convert_y(y_test_rs)
y_p = convert_y(y_pred)

In [ ]:
%run eval_score.ipynb

In [ ]:
LABELS = list(label_dict.values())

In [ ]:
show_conf_matrix(y_t, y_p, LABELS)

In [ ]:
show_clf_report(y_t, y_p, LABELS)

In [ ]:
base_model = VGG19(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

for layer in base_model.layers:
    layer.trainable = False
base_model.summary()

# score = history.evaluate(X_test,y_test,verbose=2)
print('Baseline Error: %.2f%%' %(100-score[1]*100))